In [1]:
import random
import json
import pandas as pd
import os
import json

In [6]:
csv_file_path = "/Users/meetraval/Desktop/data/finance/Finance_Final_Data.csv"

model_output_path = "/Users/meetraval/Desktop/data/finance/processed/deepseek"


In [7]:
df = pd.read_csv(csv_file_path)

In [9]:
# Path to the folder containing the .txt files

# Create a list to store the contents in order of df index
new_column_data = []

for idx in df.index:
    file_path = os.path.join(model_output_path, f"{idx}.txt")
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            content_lst = f.read().strip().split("```json")
            if len(content_lst) > 1:
              content = json.loads(content_lst[1].split("```")[0].replace("\n", ""))
            else:
              try:
                content = json.loads(content_lst[0])
              except Exception as e:
                content_1 = """{\n  "topics":""" + content_lst[0].split("""{\n  "topics":""")[1]
                content = json.loads(content_1.replace("\n", ""))
            new_column_data.append(content)
    else:
        new_column_data.append(None)  # or '' or np.nan



In [10]:
print(len(new_column_data))
print(len(df['Conversation'].to_list()))


2011
2011


In [11]:
# Add as a new column
df['predicted_response'] = new_column_data

In [12]:
import ast
# Clean json responses
topics_pred, topics_true, sents_pred, sents_true = [], [], [], []
for ii in range(len(df)):
    json_true = ast.literal_eval(df['Filename_Json'][ii])
    json_pred = df['predicted_response'][ii]
    topic_true = list(json_true.keys())
    sent_true = list(json_true.values())
    topic_pred = [x['topic'] for x in json_pred['topics']]
    sent_pred = [x['sentiment'] for x in json_pred['topics']]
    topics_pred.append(topic_pred)
    topics_true.append(topic_true)
    sents_pred.append(sent_pred)
    sents_true.append(sent_true)

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import multilabel_confusion_matrix

# Step 1: Binarize
mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(topics_true)
y_pred_bin = mlb.transform(topics_pred)

# Step 2: Compute confusion matrices
conf_matrices = multilabel_confusion_matrix(y_true_bin, y_pred_bin)

# Step 3: Print with labels
for label, matrix in zip(mlb.classes_, conf_matrices):
    print(f"Label '{label}':\n{matrix}\n")


Label 'Asset Allocation Strategy':
[[1719  130]
 [  12  150]]

Label 'Cash Flow Management':
[[1669   90]
 [   0  252]]

Label 'Creditworthiness Evaluation':
[[1711    4]
 [   2  294]]

Label 'Debt-to-Equity Ratio Assessment':
[[1784    2]
 [   2  223]]

Label 'Financial Statement Analysis':
[[1777   15]
 [   7  212]]

Label 'Forecasting and Projections':
[[1725   44]
 [  22  220]]

Label 'Interest Rate Impact Analysis':
[[1773   12]
 [   7  219]]

Label 'Internal Controls and Governance':
[[1713    7]
 [   1  290]]

Label 'Investment Performance Review':
[[1722  104]
 [   0  185]]

Label 'Legal and Contractual Obligations':
[[1678    3]
 [   8  322]]

Label 'Liquidity Risk Evaluation':
[[1732   30]
 [  12  237]]

Label 'Market Conditions Impact':
[[1405  400]
 [   2  204]]

Label 'Negotiation of Financial Terms':
[[1689   22]
 [   2  298]]

Label 'Portfolio Diversification Review':
[[1644  208]
 [   5  154]]

Label 'Regulatory Compliance Check':
[[1655   52]
 [   0  304]]

Label 'Risk

/Users/meetraval/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:875: UserWarning: unknown class(es) ['Debt-to-Eity Ratio Assessment', 'Negot Financial Terms', 'bt-to-Equity Ratio Assessment'] will be ignored
  warnings.warn(


In [14]:
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)

# --- 1. Per-Class & Aggregate Report
print(classification_report(y_true_bin, y_pred_bin, target_names=mlb.classes_))

# --- 2. Subset Accuracy (exact match)
subset_acc = accuracy_score(y_true_bin, y_pred_bin)
print(f"Subset Accuracy: {subset_acc:.4f}")

# --- 3. Micro-Averaged Scores
print("Micro Precision:", precision_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro Recall:", recall_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='micro'))

# --- 4. Macro-Averaged Scores
print("Macro Precision:", precision_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro Recall:", recall_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='macro'))

# --- 5. Weighted-Averaged Scores
print("Weighted Precision:", precision_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted Recall:", recall_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted F1 Score:", f1_score(y_true_bin, y_pred_bin, average='weighted'))

                                   precision    recall  f1-score   support

        Asset Allocation Strategy       0.54      0.93      0.68       162
             Cash Flow Management       0.74      1.00      0.85       252
      Creditworthiness Evaluation       0.99      0.99      0.99       296
  Debt-to-Equity Ratio Assessment       0.99      0.99      0.99       225
     Financial Statement Analysis       0.93      0.97      0.95       219
      Forecasting and Projections       0.83      0.91      0.87       242
    Interest Rate Impact Analysis       0.95      0.97      0.96       226
 Internal Controls and Governance       0.98      1.00      0.99       291
    Investment Performance Review       0.64      1.00      0.78       185
Legal and Contractual Obligations       0.99      0.98      0.98       330
        Liquidity Risk Evaluation       0.89      0.95      0.92       249
         Market Conditions Impact       0.34      0.99      0.50       206
   Negotiation of Financ

In [15]:
# Step 1: Binarize
mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(sents_true)
y_pred_bin = mlb.transform(sents_pred)

# Step 2: Compute confusion matrices
conf_matrices = multilabel_confusion_matrix(y_true_bin, y_pred_bin)

# Step 3: Print with labels
for label, matrix in zip(mlb.classes_, conf_matrices):
    print(f"Label '{label}':\n{matrix}\n")

Label 'negative':
[[ 471    3]
 [  92 1445]]

Label 'neutral':
[[490 134]
 [434 953]]

Label 'positive':
[[ 177  298]
 [   0 1536]]



/Users/meetraval/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:875: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(


In [16]:
# --- 1. Per-Class & Aggregate Report
print(classification_report(y_true_bin, y_pred_bin, target_names=mlb.classes_))

# --- 2. Subset Accuracy (exact match)
subset_acc = accuracy_score(y_true_bin, y_pred_bin)
print(f"Subset Accuracy: {subset_acc:.4f}")

# --- 3. Micro-Averaged Scores
print("Micro Precision:", precision_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro Recall:", recall_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='micro'))

# --- 4. Macro-Averaged Scores
print("Macro Precision:", precision_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro Recall:", recall_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='macro'))

# --- 5. Weighted-Averaged Scores
print("Weighted Precision:", precision_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted Recall:", recall_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted F1 Score:", f1_score(y_true_bin, y_pred_bin, average='weighted'))

              precision    recall  f1-score   support

    negative       1.00      0.94      0.97      1537
     neutral       0.88      0.69      0.77      1387
    positive       0.84      1.00      0.91      1536

   micro avg       0.90      0.88      0.89      4460
   macro avg       0.90      0.88      0.88      4460
weighted avg       0.90      0.88      0.89      4460
 samples avg       0.91      0.88      0.88      4460

Subset Accuracy: 0.6280
Micro Precision: 0.9004348821240559
Micro Recall: 0.8820627802690583
Micro F1 Score: 0.891154151092989
Macro Precision: 0.9040555797350337
Macro Recall: 0.8757458614763575
Macro F1 Score: 0.8833863974816308
Weighted Precision: 0.9049896916763274
Weighted Recall: 0.8820627802690583
Weighted F1 Score: 0.8871796553629027
